<a href="https://colab.research.google.com/github/estskyway/project_data_analystics/blob/main/codes/LDA/project_review_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
pip install python-mecab-ko pyLDAvis

In [97]:
import numpy as np

### 전처리


In [98]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2023pythons/datasets/DB_Total.csv')
df

,Unnamed: 0,name,reviews,stars
0,0,Google 사용자,입장방식은 변경됐는데 어플은 여전히 바코드만 보여집니다. 짐에서는 더이상 바코드스캔...,별표 5개 만점에 1개를 받았습니다.
1,1,Google 사용자,정말 답답하네요 하나. GPS 100프로 신뢰하나요? 시설선택목록에서 안보임 4월말...,별표 5개 만점에 1개를 받았습니다.
2,2,Google 사용자,입장하려다 시설검색이 안되서 결국 다른곳에 입장했습니다. 기존에 잘 쓰던 앱이 업데...,별표 5개 만점에 1개를 받았습니다.
3,3,한반도,평소 이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...,별표 5개 만점에 5개를 받았습니다.
4,4,나수인,아이코젠 덕분에 운동이 더 즐겁고 꾸준하게 하게 되었어요. 사용기록이 꼼꼼히 남고 ...,별표 5개 만점에 5개를 받았습니다.
...,...,...,...,...
2873,2873,Google 사용자,좋아요,NaN
2874,2874,Google 사용자,좋아용~!,NaN
2875,2875,Google 사용자,좋아용,NaN
2876,2876,Say Cheese,너무느려요,NaN


In [99]:
df = df.drop(columns=['Unnamed: 0'])
df

,name,reviews,stars
0,Google 사용자,입장방식은 변경됐는데 어플은 여전히 바코드만 보여집니다. 짐에서는 더이상 바코드스캔...,별표 5개 만점에 1개를 받았습니다.
1,Google 사용자,정말 답답하네요 하나. GPS 100프로 신뢰하나요? 시설선택목록에서 안보임 4월말...,별표 5개 만점에 1개를 받았습니다.
2,Google 사용자,입장하려다 시설검색이 안되서 결국 다른곳에 입장했습니다. 기존에 잘 쓰던 앱이 업데...,별표 5개 만점에 1개를 받았습니다.
3,한반도,평소 이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...,별표 5개 만점에 5개를 받았습니다.
4,나수인,아이코젠 덕분에 운동이 더 즐겁고 꾸준하게 하게 되었어요. 사용기록이 꼼꼼히 남고 ...,별표 5개 만점에 5개를 받았습니다.
...,...,...,...
2873,Google 사용자,좋아요,NaN
2874,Google 사용자,좋아용~!,NaN
2875,Google 사용자,좋아용,NaN
2876,Say Cheese,너무느려요,NaN


### reviews에 대한 결측치 확인

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2878 entries, 0 to 2877
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     2878 non-null   object
 1   reviews  2878 non-null   object
 2   stars    2118 non-null   object
dtypes: object(3)
memory usage: 67.6+ KB


In [101]:
df['reviews'].isnull().sum()
df['reviews']

0       입장방식은 변경됐는데 어플은 여전히 바코드만 보여집니다. 짐에서는 더이상 바코드스캔...
1       정말 답답하네요 하나. GPS 100프로 신뢰하나요? 시설선택목록에서 안보임 4월말...
2       입장하려다 시설검색이 안되서 결국 다른곳에 입장했습니다. 기존에 잘 쓰던 앱이 업데...
3       평소 이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...
4       아이코젠 덕분에 운동이 더 즐겁고 꾸준하게 하게 되었어요. 사용기록이 꼼꼼히 남고 ...
                              ...                        
2873                                                  좋아요
2874                                                좋아용~!
2875                                                  좋아용
2876                                                너무느려요
2877                                                아주좋아여
Name: reviews, Length: 2878, dtype: object

### 자연어 전처리

#### 유사어 대치

In [102]:
replace_list = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/2023pythons/datasets/replace_concat.xlsx.xlsx의 사본')
replace_list[40:50]

,before_replacement,after_replacement
40,핏데이,사이트
41,엡,사이트
42,옙,사이트
43,앱,사이트
44,가맹점,운동시설
45,쎈타,운동시설
46,매장,운동시설
47,휘트니스,운동시설
48,센터,운동시설
49,스포츠센터,운동시설


In [103]:
from mecab import MeCab
mecab = MeCab()
def replace_word(reviews):
    replaced_review = reviews
    for i in range(len(replace_list['before_replacement'])):
        try:
            if replace_list['before_replacement'][i].lower() in replaced_review.lower():
                replaced_review = replaced_review.replace(replace_list['before_replacement'][i], replace_list['after_replacement'][i])
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")
    return replaced_review


In [104]:
# def replace_word(tokenized_review):
#   replaced_review = str()
#   for i in range(len(replace_list['before_replacement'])):
#     try:
#       # 치환할 단어가 있는 경우에만 데이터 치환 수행
#       if replace_list['before_replacement'][i] in tokenized_review:
#         replaced_review = tokenized_review.replace(replace_list['before_replacement'][i], replace_list['after_replacement'][i])
#       else :
#         replaced_review = tokenized_review
#     except Exception as e:
#       print(f"Error 발생 / 에러명: {e}")
#       replaced_review = tokenized_review
#   return replaced_review

In [105]:
replace_word('입장 시설 검색 곳 입장 기존 앱 업데이트 후 로그인 위치 자동 검색 시설 수동 검색 후 입장 기능 불편 덕분 앱 번 삭제 데 이타 예전 앱')

'입장 시설 검색 곳 입장 기존 사이트 개선 후 로그인 위치 자동 검색 시설 수동 검색 후 입장 기능 불편 덕분 사이트 번 삭제 데 이타 예전 사이트'

In [106]:
df['replaced_review'] = df['reviews'].apply(replace_word)
df[:5]

,name,reviews,stars,replaced_review
0,Google 사용자,입장방식은 변경됐는데 어플은 여전히 바코드만 보여집니다. 짐에서는 더이상 바코드스캔...,별표 5개 만점에 1개를 받았습니다.,입장방식은 수정됐는데 사이트은 여전히 바코드만 보여집니다. 운동시설에서는 더이상 바...
1,Google 사용자,정말 답답하네요 하나. GPS 100프로 신뢰하나요? 시설선택목록에서 안보임 4월말...,별표 5개 만점에 1개를 받았습니다.,정말 답답하네요 하나. GPS 100프로 신뢰하나요? 시설선택목록에서 안보임 4월말...
2,Google 사용자,입장하려다 시설검색이 안되서 결국 다른곳에 입장했습니다. 기존에 잘 쓰던 앱이 업데...,별표 5개 만점에 1개를 받았습니다.,입장하려다 시설검색이 안되서 결국 다른곳에 입장했습니다. 기존에 잘 쓰던 사이트이 ...
3,한반도,평소 이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...,별표 5개 만점에 5개를 받았습니다.,평소 이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...
4,나수인,아이코젠 덕분에 운동이 더 즐겁고 꾸준하게 하게 되었어요. 사용기록이 꼼꼼히 남고 ...,별표 5개 만점에 5개를 받았습니다.,아이코젠 덕분에 운동이 더 즐겁고 꾸준하게 하게 되었어요. 사용기록이 꼼꼼히 남고 ...


#### 불용어 처리

In [107]:
df_stopwords = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/2023pythons/datasets/불용어concat.xlsx')
df_stopwords[:5]

,stopword
0,가
1,가까스로
2,가량
3,가령
4,가민


In [108]:
stopwords = df_stopwords['stopword'].values
# 토크나이징 함수 정의(한자 이상 적용. 불용어 삭제)
def tokenizer(replaced_review):
    result_list = list()
    for word in mecab.nouns(replaced_review) :
      if word not in stopwords:
          result_list.append(word)
    # 일부 경우(토큰 갯수 제한)
    # if len(result_list) <= 3 or len(result_list) < 12 :
    #   result_list
    return ' '.join(result_list) # return str

In [109]:
tokenizer('개의치않고 관계없이 거니와 대부분 것 바코드 스캔 건가요')

'개의 거니 바코드 스캔'

In [110]:
df['tokenized_review'] = df['replaced_review'].apply(tokenizer)
df[:5]

,name,reviews,stars,replaced_review,tokenized_review
0,Google 사용자,입장방식은 변경됐는데 어플은 여전히 바코드만 보여집니다. 짐에서는 더이상 바코드스캔...,별표 5개 만점에 1개를 받았습니다.,입장방식은 수정됐는데 사이트은 여전히 바코드만 보여집니다. 운동시설에서는 더이상 바...,입장 방식 수정 사이트 바코드 운동 시설 바코드 스캔 사이트 입장 사용료 상황 전 ...
1,Google 사용자,정말 답답하네요 하나. GPS 100프로 신뢰하나요? 시설선택목록에서 안보임 4월말...,별표 5개 만점에 1개를 받았습니다.,정말 답답하네요 하나. GPS 100프로 신뢰하나요? 시설선택목록에서 안보임 4월말...,프로 신뢰 시설 선택 목록 말 개선 문제 은행 로그인 타임아웃 적용 불편 감수 개선...
2,Google 사용자,입장하려다 시설검색이 안되서 결국 다른곳에 입장했습니다. 기존에 잘 쓰던 앱이 업데...,별표 5개 만점에 1개를 받았습니다.,입장하려다 시설검색이 안되서 결국 다른곳에 입장했습니다. 기존에 잘 쓰던 사이트이 ...,입장 시설 검색 곳 입장 기존 사이트 개선 후 로그인 위치 자동 검색 시설 수동 검...
3,한반도,평소 이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...,별표 5개 만점에 5개를 받았습니다.,평소 이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...,이동 장소 운동 걸 합리 코젠 제휴 업체 추가 업체 방문 게 낙 입장 방식 바코드 ...
4,나수인,아이코젠 덕분에 운동이 더 즐겁고 꾸준하게 하게 되었어요. 사용기록이 꼼꼼히 남고 ...,별표 5개 만점에 5개를 받았습니다.,아이코젠 덕분에 운동이 더 즐겁고 꾸준하게 하게 되었어요. 사용기록이 꼼꼼히 남고 ...,아이코 젠 덕분 운동 기록 지점 출장 본가 지속 수 점 짱 라이프 코스 중 덕분 비 대박


### 한단어 처리

In [111]:
df_oneword = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/2023pythons/datasets/한단어_concat.xlsx')
df_oneword[:4]

,one_char_keyword
0,값
1,꿀
2,꿈
3,끝


In [116]:
df_tokenized_review = df['tokenized_review']
df_tokenized_review

0       입장 방식 수정 사이트 바코드 운동 시설 바코드 스캔 사이트 입장 사용료 상황 전 ...
1       프로 신뢰 시설 선택 목록 말 개선 문제 은행 로그인 타임아웃 적용 불편 감수 개선...
2       입장 시설 검색 곳 입장 기존 사이트 개선 후 로그인 위치 자동 검색 시설 수동 검...
3       이동 장소 운동 걸 합리 코젠 제휴 업체 추가 업체 방문 게 낙 입장 방식 바코드 ...
4       아이코 젠 덕분 운동 기록 지점 출장 본가 지속 수 점 짱 라이프 코스 중 덕분 비 대박
                              ...                        
2873                                                     
2874                                                     
2875                                                     
2876                                                     
2877                                                     
Name: tokenized_review, Length: 2878, dtype: object

In [142]:
onewords = df_oneword['one_char_keyword'].values
def tokenizer2(tokenized_review):
    oneword_result_list = list()
    for word in mecab.nouns(tokenized_review):
        # 대소문자 구분을 무시하고 1글자 키워드를 제거
        if len(word) > 1 or word.lower() in onewords:
            oneword_result_list.append(word)
    # 명사가 없으면 None 반환
    if not oneword_result_list:
        return None
    return ' '.join(oneword_result_list)  # return str

In [137]:
# onewords = df_oneword['one_char_keyword'].values
# def tokenizer2(tokenized_review):
#     oneword_result_list = list()
#     for word in mecab.nouns(tokenized_review) :
#       if len(word) > 0 and word not in onewords :
#         oneword_result_list.append(word)
#     return ' '.join(oneword_result_list) # return str

In [143]:
tokenizer2('아이코 젠 덕분 운동 기록 지점 출장 본가 지속 수 점 짱 라이프 코스 중 덕분 비 대박')

'아이코 덕분 운동 기록 지점 출장 본가 지속 수 라이프 코스 덕분 대박'

In [144]:
df['oneword_review'] = df['tokenized_review'].apply(tokenizer2)
df[:7]

,name,reviews,stars,replaced_review,tokenized_review,oneword_review
0,Google 사용자,입장방식은 변경됐는데 어플은 여전히 바코드만 보여집니다. 짐에서는 더이상 바코드스캔...,별표 5개 만점에 1개를 받았습니다.,입장방식은 수정됐는데 사이트은 여전히 바코드만 보여집니다. 운동시설에서는 더이상 바...,입장 방식 수정 사이트 바코드 운동 시설 바코드 스캔 사이트 입장 사용료 상황 전 ...,입장 방식 수정 사이트 바코드 운동 시설 바코드 스캔 사이트 입장 사용료 상황 일자...
1,Google 사용자,정말 답답하네요 하나. GPS 100프로 신뢰하나요? 시설선택목록에서 안보임 4월말...,별표 5개 만점에 1개를 받았습니다.,정말 답답하네요 하나. GPS 100프로 신뢰하나요? 시설선택목록에서 안보임 4월말...,프로 신뢰 시설 선택 목록 말 개선 문제 은행 로그인 타임아웃 적용 불편 감수 개선...,프로 신뢰 시설 선택 목록 개선 문제 은행 로그인 타임아웃 적용 불편 감수 개선 적...
2,Google 사용자,입장하려다 시설검색이 안되서 결국 다른곳에 입장했습니다. 기존에 잘 쓰던 앱이 업데...,별표 5개 만점에 1개를 받았습니다.,입장하려다 시설검색이 안되서 결국 다른곳에 입장했습니다. 기존에 잘 쓰던 사이트이 ...,입장 시설 검색 곳 입장 기존 사이트 개선 후 로그인 위치 자동 검색 시설 수동 검...,입장 시설 검색 입장 기존 사이트 개선 로그인 위치 자동 검색 시설 수동 검색 입장...
3,한반도,평소 이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...,별표 5개 만점에 5개를 받았습니다.,평소 이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...,이동 장소 운동 걸 합리 코젠 제휴 업체 추가 업체 방문 게 낙 입장 방식 바코드 ...,이동 장소 운동 합리 코젠 제휴 업체 추가 업체 방문 입장 방식 바코드 편의 개선 ...
4,나수인,아이코젠 덕분에 운동이 더 즐겁고 꾸준하게 하게 되었어요. 사용기록이 꼼꼼히 남고 ...,별표 5개 만점에 5개를 받았습니다.,아이코젠 덕분에 운동이 더 즐겁고 꾸준하게 하게 되었어요. 사용기록이 꼼꼼히 남고 ...,아이코 젠 덕분 운동 기록 지점 출장 본가 지속 수 점 짱 라이프 코스 중 덕분 비 대박,아이코 덕분 운동 기록 지점 출장 본가 지속 수 라이프 코스 덕분 대박
5,최경진,"작년부터 계속 이용하고 있는 아이코젠, 정말 좋아요. 축 처져있는 여름날씨에도 운동...",별표 5개 만점에 5개를 받았습니다.,"작년부터 계속 사용하고 있는 아이코젠, 정말 좋아요. 축 처져있는 여름날씨에도 운동...",작년 코젠 날씨 운동 사이트 편리 내역 내 정확 수 코로나 시국 편리 운동 수,작년 코젠 날씨 운동 사이트 편리 내역 정확 수 코로나 시국 편리 운동 수
6,이루이루,오랜기간 사용중인데 아이코젠 회원가입 덕분에 꾸준히 운동하고 있습니다. 개인적으로 ...,별표 5개 만점에 5개를 받았습니다.,오랜기간 사용중인데 아이코젠 회원회원가입 덕분에 꾸준히 운동하고 있습니다. 개인적으...,기간 중 코젠 회원 회원 가입 덕분 운동 개인 운동 장기간 운동 거 초기 오류 안정...,기간 코젠 회원 회원 가입 덕분 운동 개인 운동 장기간 운동 초기 오류 안정 사이트...
